In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import sys
import matplotlib.pyplot as plt

ROOT = "./"
sys.path.append(ROOT + 'lib/')

from helpers import *
from general_trainer import compete


SyntaxError: invalid syntax (238538248.py, line 10)

# Testing / Toy Model

In [ ]:
class bigboy(nn.Module):
  def __init__(self, out):
    super().__init__()
    self.av = nn.ReLU()

    self.to_one = nn.Parameter(torch.zeros(6))

    self.fc1 = nn.Linear(350, 200)
    self.fc2 = nn.Linear(200, 100)
    self.fc3 = nn.Linear(100, 10)



    self.fc4 = nn.Linear(10, out)

    self.prob = nn.Softmax(dim=1)

  def forward(self, x):

    # x = torch.einsum("bij,j->bi", x, self.to_one)
    x = x[:, :, 0]
    T = x.shape[1]

    pad = torch.zeros(x.shape[0], 350 - T).to(device)

    padded = torch.cat((x, pad), dim=1)


    current = padded

    for f in (self.fc1, self.fc2, self.fc3, self.fc4):
      current = self.av(f(current))

    return self.prob(current)








In [ ]:
class PseudoSet(Dataset):
  def __init__(self, classes):
    self.classes = classes

    self.all = []
    for i, class_ in enumerate(self.classes):
      label = torch.zeros(len(self.classes))
      label[i] = 1
      for ex in class_:
        self.all.append((ex, label))


  def __getitem__(self, idx):
    return self.all[idx]

  def __len__(self):
    return len(self.all)

with open(ROOT + "datasets/toy_data_train.pt", "rb") as f:
  toydata_train = torch.load(f, map_location=device)
with open(ROOT + "datasets/toy_data_valid.pt", "rb") as f:
  toydata_valid = torch.load(f, map_location=device)

toytrain = DataLoader(toydata_train, batch_size=len(toydata_train), collate_fn=padded_collate, shuffle=True)
toyvalid = DataLoader(toydata_valid, batch_size=len(toydata_valid), collate_fn=padded_collate, shuffle=True)

In [ ]:
ex = toydata_train[100][0].cpu()
plt.scatter(ex[:, -2], ex[:, 0])
print(toydata_train[200][1])

In [ ]:
# model = FluxAnomalyPredictionLSTM(15, 0.15, residual=0, features=6, out=3).to(device)
# model = lstm(6, 20, 3).to(device)
model = bigboy(3).to(device)


optim = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=10**-6.5)
loss_fn = nn.CrossEntropyLoss().to(device)

trainers = [
    ("Flux Anomaly Predictor", model, optim)
]

compete(trainers, 150, toytrain, toyvalid)
